In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
# Use the scipy routine "solve_ivp" for our numerical
# integrations.
from scipy.integrate import solve_ivp

# Rigid Body Motion

Let us study the rotation of a rigid body. Recall the definitions of the angular velocity vector $\boldsymbol\omega$, the moment of inertia tensor $I_{ij} = \int d^3\mathbf{r}\:\rho(\mathbf r)(r^2\delta_{ij}-r_ir_j)$, and angular momentum vector $\mathbf L=I\cdot \boldsymbol\omega$. Recall that the torque determines the rate of change of angular momentum $\boldsymbol\tau = \dot{\mathbf L}$.

Let's analyse the system in the principal axis in the body-frame in which the moment of inertia is diagonal, i.e.
$$I=\begin{pmatrix}I_1&0&0\\0&I_2&0\\0&0&I_3\end{pmatrix}.$$
Let's call the basis vectors $\mathbf e_1$, $\mathbf e_2$, and $\mathbf e_3$. Don't forget that $\mathbf e_i$ change with time!
In this basis, the angular momentum is $\mathbf L = I_1 \omega_1\mathbf e_1 + I_2\omega_2\mathbf e_2 + I_3\omega_3\mathbf e_3$. In the absence of torques, the Euler equations state,
\begin{align}
I_1\dot\omega_1&=(I_2-I_3)\omega_2\omega_3,\\
I_2\dot\omega_2&=(I_3-I_1)\omega_1\omega_3,\\
I_3\dot\omega_3&=(I_1-I_2)\omega_1\omega_2.
\end{align}
These equations can be derived by saying $\dot{\mathbf{L}}=0$, including the $\dot{\mathbf{e}}_i$ terms and using $\dot{\mathbf v}=\boldsymbol\omega\times\mathbf{v}$ for a vector $\mathbf{v}$ rotating with angular velocity $\mathbf{\omega}$.

Fill in the following function for the derivatives of $\mathbf{\omega}$.  We will use this, along with the ```scipy.integrate``` function ```solve_ivp```, to numerically integrate the Euler equations.  Later on we will want to evolve things fairly accurately, so we'll use ```method='DOP853'``` in ```solve_ivp``` since that's good for high accuracy.

In [ ]:
def euler_derivs(t,y,I1,I2,I3):
    """The derivatives of the angular velocities ordered as
    vec{y} = (omega1, omega2, omega3).  All quantities are
    in the "body frame", in the eigenbasis of the inertia tensor.
    Ij are the three principal moments of inertia."""
    dy    = np.empty_like(y)
    dy[0] = ...
    dy[1] = ...
    dy[2] = ...
    return(dy)

Hint: You can use the ```args``` keyword in ```solve_ivp``` to pass the "extra" arguments (I1, I2 and I3) to ```euler_derivs```.

What happens when $\boldsymbol\omega_0$ is aligned with one of the principal moments of inertia? Check that your function returns what you expect in this situation. 

What happens when $I_1=I_2=I_3$? Again, check that your function makes sense.

(a) Show analytically that if two of the principal moments are equal, then $\omega$ precesses around the other principal moment.  Label the unique moment $I_1$, and the others $I=I_2=I_3$.
If $I_1>I$, we call the object oblate, and $I_1$ is in the short dimension of e.g. a coin. If $I_1<I$, we call the object prolate, and $I_1$ is in the long dimension of e.g. an American football. 

Analytically compute the angular velocity of precession $\omega_p=2\pi/T_p$ in terms of the parameters already defined.

(b) Use your numerical solution to estimate the precession rate $\omega_p$ of an object with $I_1=2$, $I=1$, and $\boldsymbol\omega = (1,1,1)^T$ (e.g. you can find the first non-zero time when $|\boldsymbol\omega-\boldsymbol\omega_0|<\epsilon$ for some $\epsilon\ll1$). Compare with theoretical expectations.

In [ ]:
times = np.linspace(0,10,2000)
omega0= [1,1,1]     # Initial angular velocity.
II    = (2.,1.,1.)  # Principal moments of inertia.
#
# FILL THIS IN

Recall than in torque-free motion the energy is given by the kinetic energy,
$T=\frac{1}{2}\boldsymbol\omega\cdot I\cdot\boldsymbol\omega.$
Fill in the following function that calculates the energy from a given $\boldsymbol\omega$ and $I$ in the body frame.

In [ ]:
def energy(omega,II):
    """
    omega: angular velocity vector, shape (3,)
    II: principal moments of inertia, shape (3,)
    returns: energy, scalar
    """
    return( SOMETHING )

Check to see how well energy is conserved for some random trajectories (i.e. random initial $\mathbf{\omega}_0$) if $I=(2,1,0.5)$.

(c) In $\boldsymbol\omega$-space, what shape do the energy contours take? Recall that since energy is conserved, $\boldsymbol\omega$ is constrained to be on such surfaces.

The following function will plot multiple trajectories overlaid on the allowed energy surface in 3D.

In [ ]:
def plot_trajectories(trajectories,II):
    """
    trajectories: array of shape (M,3,N) containing M omega vectors at N time-steps
                (all omega vectors assumed to be at the same energy)
    II: principal moments of inertia
    returns nothing, plots trajectories as lines in 3d plot overlaid on energy 
                surface (using energy of first omega vector of first trajectory)
    """
    N = 20
    theta, phi = np.linspace(0,np.pi,N),np.linspace(0,2*np.pi,2*N)
    theta, phi = np.meshgrid(theta, phi) #surface is 2d, need 2d array of points
    
    T = energy(trajectories[0,:,0],II)
    print("Energy for trajectory 0 at time 0 is ",T)

    x = np.sin(theta)*np.cos(phi)*(2*T/II[0])**.5
    y = np.sin(theta)*np.sin(phi)*(2*T/II[1])**.5
    z = np.cos(theta)*(2*T/II[2])**.5

    fig = plt.figure(figsize=(8,8))
    ax  = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x,y,z,alpha=.2,facecolors=[["w"]*N]*2*N)
    ax.set_xlabel("$\omega_1$")
    ax.set_ylabel("$\omega_2$")
    ax.set_zlabel("$\omega_3$")
    
    bound = np.amax([x,y,z])
    ax.set_xlim(-bound,bound)
    ax.set_ylim(-bound,bound)
    ax.set_zlim(-bound,bound)
    
    for omegas in trajectories:
        ax.plot(*omegas)  # was omegas.T

The following plots the trajectories of some randomly chosen initial $\boldsymbol\omega$s.

In [ ]:
def get_random_initials(II,energy=1,n=30):
    """
    II: principal moments of inertia, scalars
    energy: energy of initial states
    n: number of points to sample
    returns: n randomly chosen omega vectors with given energy
    """
    randoms = np.zeros((n,3))
    for i in range(n): #sample uniformly from sphere using rejection
        x = np.random.rand(3)*2-1
        r = np.sum(x**2)
        while r > 1:
            x = np.random.rand(3)*2-1
            r = np.sum(x**2)
        randoms[i] = x/r**.5
    randoms[:,0] = randoms[:,0]*(2*energy/II[0])**.5
    randoms[:,1] = randoms[:,1]*(2*energy/II[1])**.5
    randoms[:,2] = randoms[:,2]*(2*energy/II[2])**.5
    return(randoms)


II      = np.array([2,1,0.5])
omega0s = get_random_initials(II)
times   = np.linspace(0,20,2000)
trajs   = np.zeros( (omega0s.shape[0],3,times.size) )
for i in range(omega0s.shape[0]):
    res   = solve_ivp(euler_derivs,[times[0],times[-1]],omega0s[i,:],\
                                    method='DOP853',t_eval=times,args=II)
    trajs[i,:,:] = res.y[:,:]
plot_trajectories(trajs,II)
plt.tight_layout()

(d) The Earth's axis of rotation precesses with a period of about 430 days (not to be confused with the precession of 26,000 years around its orbital rotation axis, which is caused mostly by tidal/gravitational forces). This precession is known as _Chandler wobble_. Assuming the Earth is a rigid oblate spherioid (i.e. $I_1>I$), estimate the fractional asymmetry of the principal moments of inertia (i.e. $\frac{I-I_1}{I}$). From this, assuming the Earth's density is a scaled spherically symmetric distribution (like how we plotted the energy ellipsoid surfaces above), calculate its _ellipticity_ (ratio of major to minor axes).

Now consider the general case, $I_1>I_2>I_3$ (in the approriate basis). We know that the principal moments of inertia are fixed points (equilibria) for $\boldsymbol\omega$. However, are they stable? If we consider the motion very close to the principal moments (i.e. linearise the differential equation) we find two are stable and one is unstable. There are no other equilibria.

### Proof ###

Consider the equilibrium at principal moment $1$.  Assuming $\omega_2$ and $\omega_3$ are small ($\mathcal{O}(\epsilon)$), we find that $\dot\omega_1=\mathcal{O}(\epsilon^2)$, and thus
$$
  \frac{d}{dt} \begin{pmatrix}\omega_2\\\omega_3\end{pmatrix}
  =\omega_1\begin{pmatrix}0&\frac{I_k-I_i}{I_j}\\\frac{I_i-I_j}{I_k}&0\end{pmatrix}
  \begin{pmatrix}\omega_2\\\omega_3\end{pmatrix}+\mathcal{O}(\epsilon^2).
$$
We can in fact generalize this to any cyclic ordered triple $ijk$, of which $i=1$, $j=2$ and $k=3$ is just one example.

The matrix has eigenvalues $\lambda_\pm=\pm\omega_i\sqrt{-(I_i-I_k)(I_i-I_j)/(I_jI_k)}$.
Thus
\begin{align}
  (I_i-I_k)(I_i-I_j)>0
  &\implies\text{imaginary eigenvalues}\implies\boldsymbol\omega \text{ rotates around } \mathbf{e_i},\\
  (I_i-I_k)(I_i-I_j)<0
  &\implies\text{one positive, one negative eigenvalue}\implies\mathbf e_i \text{ is saddle point}.
\end{align}
For the largest and smallest moments, $(I_i-I_j)(I_i-I_k)>0$, so we see precession and the equilibrium is stable. The middle moment is a saddle point, so is unstable.

There are no other equilibria apart from the trivial $\boldsymbol\omega=0$.

Consider the following video of  the rotation of a free rigid body taken on the ISS. Note that the initial angular velocity vector is very close to a principal moment of inertia.

In [ ]:
from IPython.display import Video
Video("Dancing T-handle in zero-g.mp4",width=700)

(e) What can you conclude about the values of the principal moments of inertia, and specifically the moment that it starts off close to?

### Intertial frames

Now let's transform to an inertial reference frame, i.e. one that isn't fixed on the body, and denote quantities in this (lab) frame using primes ('), like we've been doing in lecture.

Let's define the rotation between the two frames to be $R(t)$ according to
$\mathbf x'=R\cdot\mathbf x$
for any vector $\mathbf x$, i.e. $R(t)$ takes us from the body-frame vector to the lab/intertial frame vector.

(f) If at time $t$ the body frame is rotating with angular velocity $\mathbf{\omega}$, show that
$$
  \dot R_{ij}=R_{ik}\epsilon_{klj}\omega_l.
$$
(Hint: what is $\dot{\mathbf x'}$ for $\mathbf x$ fixed in the body frame, first expressed in terms of $R$ and second in terms of $\mathbf{\omega}\times$?)

Let us write a routine to return the dual of the 3-vector $\omega$, i.e. $\epsilon_{ijk}\omega_k$, then we can always integrate our ODE for $R(t)$.

In [ ]:
def hodge_dual(w):
    """Returns the Hodge dual of 3-vector w."""
    # There is a clever way to do this using numpy's "einsum"
    # routine, but for clarity let's just type them all out ...
    Omega      = np.zeros( (3,3) )
    Omega[0,1] = -w[2]
    Omega[1,2] = -w[0]
    Omega[2,0] = -w[1]
    Omega[1,0] = -Omega[0,1]
    Omega[2,1] = -Omega[1,2]
    Omega[0,2] = -Omega[2,0]
    return(Omega)

Complete the routine below to return the derivatives of $\omega$ (in the body frame) and the derivatives of the $R$ matrix.  We will simply pack them into our "$y$" vector as $(\omega_1,\omega_2,\omega_3,R_{11},R_{12},\cdots)$.

In [ ]:
def euler_derivs(t,y,I1,I2,I3):
    """Extend our earlier derivative routine to also
    include the derivatives of R.  We will store first
    the three components of omega (in the body frame)
    and then the 9 components of the R matrix.
    Ij are the three principal moments of inertia."""
    # Extract R from our y-vector.
    RR    = y[3:].reshape(3,3)
    # Generate space for the derivatives.
    dy    = np.empty_like(y)
    # The first 3 equations are as we had before.
    dy[0] = (I2-I3)/I1*y[1]*y[2]
    dy[1] = (I3-I1)/I2*y[0]*y[2]
    dy[2] = (I1-I2)/I3*y[0]*y[1]
    # Now compute Rdot, and unpack it into dy[3:] using flatten().
    Rdot  = SOMETHING
    dy[3:]= Rdot.flatten() 
    return(dy)

(g) Now model a coin, which has a thickness very small compared to the radius.  Analytically compute the moments of inertia assuming the "thin" direction is $\widehat{z}$.  Try using different initial angular velocities and integrate our equations to find its rotation. You may wish to throw a coin in the air a few times to get some intuition for what you expect!

Now we have an array of rotation matrices.  If we multiply $[0,1,1]$ by each matrix we get the tip of the normal vector to a coin that started at 45 degrees to the $z$-axis (in the $y-z$ plane). We should see how that rotates and tumbles, and whether it matches our expectations.

In [ ]:
# We will repurpose our plot_trajectories code, but now with a
# unit II matrix to plot the motion of the tip on a unit sphere
# (so "II" now has nothing to do with inertias).
# Also, the axis labels will still be omega_i, but that's not
# true anymore, it's really x-y-z.
#
# FINISH THIS
#

## Animation

If you're feeling so inclined, you can actually use Matplotlib to animate this rather than using these static traces.  This is left to your discretion.

## Free rotation

(h) In the previous questions, we assumed that objects rotate freely in zero-gravity, e.g. in free-fall on the ISS. We also assumed that they rotate freely in the presence of gravity when thrown in the air. However, they don't rotate freely in the presence of gravity when, e.g. resting on a table. What exactly does it mean for something to rotate freely, and where does this difference in behaviour come from?

# The End